In [7]:
import math

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import pyproj
from plotly.subplots import make_subplots
from pykalman import KalmanFilter
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler

from utils import ORBSLAMResults, umeyama_alignment


def fit_trajectory(source_points, target_points):
    assert source_points.shape == target_points.shape

    # Create and fit the MinMaxScaler for source_points and target_points
    source_scaler = MinMaxScaler(feature_range=(-1, 1))
    target_scaler = MinMaxScaler(feature_range=(-1, 1))

    scaled_source_points = source_scaler.fit_transform(source_points)
    scaled_target_points = target_scaler.fit_transform(target_points)

    layer_sizes = (256, 256)
    n_layers = len(layer_sizes)
    mlp = MLPRegressor(hidden_layer_sizes=layer_sizes,
                       activation='tanh',
                       solver='lbfgs',
                       learning_rate='adaptive',
                       max_iter=400,
                       tol=1e-4,
                       n_iter_no_change=10,
                       verbose=True,
                       random_state=42)
    # mlp.coefs_ = np.identity(n_layers-1) + np.random.normal(size=(n_layers-1,))
    # mlp.intercepts_ = np.zeros((n_layers-1,))

    mlp.fit(scaled_source_points, scaled_target_points)

    return mlp, source_scaler, target_scaler


def predict_trajectory(mlp, source_points, source_scaler, target_scaler):
    scaled_new_source_points = source_scaler.transform(source_points)
    scaled_predicted_target_points = mlp.predict(scaled_new_source_points)

    # De-normalize the predicted target points
    predicted_target_points = target_scaler.inverse_transform(scaled_predicted_target_points)

    return predicted_target_points

In [8]:


results = ORBSLAMResults("data")

gps_trajectory_wgs = pd.DataFrame([(kf.gps.lat, kf.gps.lon, kf.gps.alt)
                                   for kf in results.keyframes[1:]], columns=['lat', 'lon', 'alt'])
slam_trajectory = np.array([(kf.x, kf.y, kf.z) for kf in results.keyframes[1:]])

# Create transformers for WGS84 <-> UTM35N
wgs2utm = pyproj.Transformer.from_crs(4326, 32635)
utm2wgs = pyproj.Transformer.from_crs(32635, 4326)

# Convert GPS trajectory (WGS84) to UTM35N
gps_trajectory_utm = np.array([wgs2utm.transform(kf.gps.lat, kf.gps.lon, kf.gps.alt)
                               for kf in results.keyframes[1:]])

# Align SLAM trajectory to GPS trajectory
R, t, c = umeyama_alignment(slam_trajectory.T, gps_trajectory_utm.T, True)
aligned_slam_trajectory_utm = np.array([t + c * R @ p for p in slam_trajectory])

# Convert SLAM trajectory (UTM35N) to WGS84
aligned_slam_trajectory_wgs = pd.DataFrame([utm2wgs.transform(p[0], p[1], p[2])
                                           for p in aligned_slam_trajectory_utm], columns=['lat', 'lon', 'alt'])

model, source_scaler, target_scaler = fit_trajectory(aligned_slam_trajectory_utm, gps_trajectory_utm)

fitted_slam_trajectory_utm = predict_trajectory(model, aligned_slam_trajectory_utm, source_scaler, target_scaler)
fitted_slam_trajectory_wgs = pd.DataFrame([utm2wgs.transform(p[0], p[1], p[2])
                                          for p in fitted_slam_trajectory_utm], columns=['lat', 'lon', 'alt'])

slam_estimates = np.array([(e.lat, e.lon, e.alt) for e in results.slam_estimates])
aligned_slam_estimate_utm = np.array([t + c * R @ p for p in slam_estimates])
fitted_slam_estimate_utm = predict_trajectory(model, aligned_slam_estimate_utm, source_scaler, target_scaler)
fitted_slam_estimate_wgs = pd.DataFrame([utm2wgs.transform(p[0], p[1], p[2])
                                        for p in fitted_slam_estimate_utm], columns=['lat', 'lon', 'alt'])

fig = go.Figure()
fig.add_trace(
    go.Scattermapbox(lat=gps_trajectory_wgs['lat'],
                     lon=gps_trajectory_wgs['lon'],
                     mode='markers+lines',
                     marker=dict(color='blue'),
                     name='GPS'))
fig.add_trace(
    go.Scattermapbox(lat=aligned_slam_trajectory_wgs['lat'],
                     lon=aligned_slam_trajectory_wgs['lon'],
                     mode='markers+lines',
                     marker=dict(color='red'),
                     name='SLAM'))
fig.add_trace(
    go.Scattermapbox(lat=fitted_slam_trajectory_wgs['lat'],
                     lon=fitted_slam_trajectory_wgs['lon'],
                     mode='markers+lines',
                     marker=dict(color='forestgreen'),
                     name='fitted SLAM'))
fig.add_trace(
    go.Scattermapbox(lat=fitted_slam_estimate_wgs['lat'],
                     lon=fitted_slam_estimate_wgs['lon'],
                     mode='markers+lines',
                     name='fitted SLAM estimate'))
fig.update_geos(projection_type="transverse mercator")
fig.update_layout(mapbox_style="open-street-map",
                  mapbox=dict(center=dict(lat=np.mean(gps_trajectory_wgs['lat']), lon=np.mean(
                      gps_trajectory_wgs['lon'])), zoom=15),
                  margin={"t": 0, "b": 0, "l": 0, "r": 0},
                  height=800)
fig.show()


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =       100099     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.20672D-01    |proj g|=  3.98542D-01

At iterate    1    f=  1.35246D-01    |proj g|=  5.03736D-01

At iterate    2    f=  4.20838D-02    |proj g|=  8.28701D-02

At iterate    3    f=  3.47664D-02    |proj g|=  2.30964D-02


 This problem is unconstrained.



At iterate    4    f=  2.91960D-02    |proj g|=  4.10142D-02

At iterate    5    f=  2.61514D-02    |proj g|=  5.98312D-02

At iterate    6    f=  2.36932D-02    |proj g|=  1.37177D-02

At iterate    7    f=  2.17571D-02    |proj g|=  2.83968D-02

At iterate    8    f=  1.96434D-02    |proj g|=  3.56486D-02

At iterate    9    f=  1.92845D-02    |proj g|=  4.23993D-02

At iterate   10    f=  1.86204D-02    |proj g|=  1.14496D-02

At iterate   11    f=  1.85625D-02    |proj g|=  1.51466D-03

At iterate   12    f=  1.85492D-02    |proj g|=  1.17317D-03

At iterate   13    f=  1.85190D-02    |proj g|=  4.29203D-03

At iterate   14    f=  1.84841D-02    |proj g|=  5.25246D-03

At iterate   15    f=  1.84373D-02    |proj g|=  6.96213D-03

At iterate   16    f=  1.84183D-02    |proj g|=  5.24935D-03

At iterate   17    f=  1.84024D-02    |proj g|=  2.39321D-03

At iterate   18    f=  1.83858D-02    |proj g|=  4.03023D-03

At iterate   19    f=  1.83576D-02    |proj g|=  7.17890D-03

At iter

/Users/erik/miniconda3/envs/gorbslam/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html

